In [8]:
import json
with open('dataset_43428_1.txt', encoding="utf-8") as json_file:
    dataset = json.load(json_file)

In [3]:
import re 
def preproc(text):
    text = re.sub(" +", " ", text)
    useless_symblos = ["\\n","\",","\\","-","—","\'","\"", "«","»","(", ")","&quot"]
    for sym in useless_symblos:
        text = text.replace(sym," ")
    text = re.sub(" +", " ", text)
    text = text.replace(" .",". ")
    text = text.replace(" ,",", ")
    text = text.replace(" ?","? ")
    text = text.replace(" !","! ")
    text = text.replace(" :",": ")
    text = text.replace(" ;","; ")
    text = re.sub(" +", " ", text)
    text = text.strip()

    return(text)

In [19]:
print('Before preprocessing: ')
print(dataset[0])

Before preprocessing: 
В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. Как сообщает Лента.ру, первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля, однако вскоре были опровергнуты. В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич. Об этом сообщает сербский телеканал B92. Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина непризнанного сербского государства, существовавшего на территории Хорватии в 1992 1995 годах. В 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности. В частности, ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 1993 годах. За информацию, способную помочь в поимке Горана Хаджича, в США была назначена на

In [10]:
for i in range(len(dataset)):
    dell = preproc(dataset[i]).split('\n')
    clean_sentence = ''
    for word in dell:
        clean_sentence = clean_sentence + word + ' '
    dataset[i] = preproc(clean_sentence)

In [11]:
print('After preprocessing: ')
print(dataset[0])

After preprocessing: 
В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. Как сообщает Лента.ру, первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля, однако вскоре были опровергнуты. В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич. Об этом сообщает сербский телеканал B92. Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина непризнанного сербского государства, существовавшего на территории Хорватии в 1992 1995 годах. В 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности. В частности, ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 1993 годах. За информацию, способную помочь в поимке Горана Хаджича, в США была назначена наг

In [15]:
import nltk
nltk.download('punkt')
from itertools import combinations
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
import networkx as nx

def similarity(s1, s2):
    if not len(s1) or not len(s2):
        return 0.0
    return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))

def summarize(text, n=3):
    sentences = sent_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    lmtzr = RussianStemmer()
    words = [set(lmtzr.stem(word) for word in tokenizer.tokenize(sentence.lower()))
             for sentence in sentences]

    pairs = combinations(range(len(sentences)), 2)
    scores = [(i, j, similarity(words[i], words[j])) for i, j in pairs]
    scores = filter(lambda x: x[2], scores)

    g = nx.Graph()
    g.add_weighted_edges_from(scores)
    pr = nx.pagerank(g)

    tr = sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr),
                  key=lambda x: pr[x[0]], reverse=True)

    top_n = sorted(tr[:n])
    return ' '.join(x[2] for x in top_n)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rauan.assabayev/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
dataset_summarized = [summarize(text) for text in dataset]

In [23]:
print("Summarized: ",dataset_summarized[0])
print("Origin: ",dataset[0])

Summarized:  В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. За информацию, способную помочь в поимке Горана Хаджича, в США была назначена награда в 5 млн долларов. После того как в мае 2011 года был арестован бывший командующий армией Республики Сербской Ратко Младич, Хаджич оставался последним преступником, выдачи которого Гаагский трибунал добивался от Сербии.
Origin:  В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. Как сообщает Лента.ру, первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля, однако вскоре были опровергнуты. В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич. Об этом сообщает сербский телеканал B92. Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская К

In [24]:
with open('output_43428_1.txt', 'w', encoding='utf-8') as f:
    json.dump(dataset_summarized, f, ensure_ascii=False, indent=4)

In [29]:
from rouge import Rouge 

rouge = Rouge()

for i in range(len(dataset_summarized)):
    if len(dataset_summarized[i]) <= 0 or len(dataset[i]) <= 0:
        continue
    scores = rouge.get_scores(dataset[i][:300], dataset_summarized[i])
    print("Scores for sentence ", i, ": ", scores, "\n")

Scores for sentence  0 :  [{'rouge-1': {'f': 0.3999999952399093, 'p': 0.5121951219512195, 'r': 0.328125}, 'rouge-2': {'f': 0.3689320340842681, 'p': 0.475, 'r': 0.30158730158730157}, 'rouge-l': {'f': 0.4285714237401083, 'p': 0.525, 'r': 0.3620689655172414}}] 

Scores for sentence  1 :  [{'rouge-1': {'f': 0.09302325083829123, 'p': 0.1, 'r': 0.08695652173913043}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.050632906399615936, 'p': 0.05263157894736842, 'r': 0.04878048780487805}}] 

Scores for sentence  2 :  [{'rouge-1': {'f': 0.41904761442902494, 'p': 0.5789473684210527, 'r': 0.3283582089552239}, 'rouge-2': {'f': 0.36893203423131304, 'p': 0.5135135135135135, 'r': 0.2878787878787879}, 'rouge-l': {'f': 0.44897958713660974, 'p': 0.5945945945945946, 'r': 0.36065573770491804}}] 

Scores for sentence  3 :  [{'rouge-1': {'f': 0.46753246254005737, 'p': 0.4864864864864865, 'r': 0.45}, 'rouge-2': {'f': 0.39999999500800004, 'p': 0.4166666666666667, 'r': 0.38461538461538464}, 'rouge-

Scores for sentence  90 :  [{'rouge-1': {'f': 0.11111110620000023, 'p': 0.1282051282051282, 'r': 0.09803921568627451}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.10389609890369395, 'p': 0.10810810810810811, 'r': 0.1}}] 

Scores for sentence  91 :  [{'rouge-1': {'f': 0.7692307644545119, 'p': 0.975609756097561, 'r': 0.6349206349206349}, 'rouge-2': {'f': 0.7647058775855441, 'p': 0.975, 'r': 0.6290322580645161}, 'rouge-l': {'f': 0.7826086908435729, 'p': 0.972972972972973, 'r': 0.6545454545454545}}] 

Scores for sentence  92 :  [{'rouge-1': {'f': 0.45652173413988656, 'p': 0.4772727272727273, 'r': 0.4375}, 'rouge-2': {'f': 0.3999999950098766, 'p': 0.4186046511627907, 'r': 0.3829787234042553}, 'rouge-l': {'f': 0.4634146291582392, 'p': 0.48717948717948717, 'r': 0.4418604651162791}}] 

Scores for sentence  93 :  [{'rouge-1': {'f': 0.21917807735034728, 'p': 0.18604651162790697, 'r': 0.26666666666666666}, 'rouge-2': {'f': 0.056338023336639974, 'p': 0.047619047619047616, 'r': 0.

Scores for sentence  163 :  [{'rouge-1': {'f': 0.7758620641795483, 'p': 0.9782608695652174, 'r': 0.6428571428571429}, 'rouge-2': {'f': 0.7719298197830102, 'p': 0.9777777777777777, 'r': 0.6376811594202898}, 'rouge-l': {'f': 0.9024390194170138, 'p': 0.9736842105263158, 'r': 0.8409090909090909}}] 

Scores for sentence  164 :  [{'rouge-1': {'f': 0.36363635900826446, 'p': 0.5, 'r': 0.2857142857142857}, 'rouge-2': {'f': 0.33333332871913585, 'p': 0.46153846153846156, 'r': 0.2608695652173913}, 'rouge-l': {'f': 0.39999999553241, 'p': 0.59375, 'r': 0.30158730158730157}}] 

Scores for sentence  165 :  [{'rouge-1': {'f': 0.07017543427516186, 'p': 0.1111111111111111, 'r': 0.05128205128205128}, 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0}, 'rouge-l': {'f': 0.06185566561377437, 'p': 0.09090909090909091, 'r': 0.046875}}] 

Scores for sentence  166 :  [{'rouge-1': {'f': 0.6238532061947648, 'p': 0.7727272727272727, 'r': 0.5230769230769231}, 'rouge-2': {'f': 0.5981308363140886, 'p': 0.7441860465116279, 'r':